In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
%cd /content/

/content


In [1]:
!pip install ultralytics deep-sort-realtime opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 135.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 134.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 101.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 42.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12


In [9]:
import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from collections import defaultdict
import numpy as np
from google.colab.patches import cv2_imshow # For displaying videos in Colab
from google.colab import files # For file uploads/downloads

# Initialize YOLOv5s and DeepSort
model = YOLO('yolov5su.pt')
tracker = DeepSort(
    max_age=150,
    n_init=10,
    embedder="mobilenet",
    embedder_gpu=True,
    half=False,
    nn_budget=200
)

# Re-ID buffer setup
reid_buffer = defaultdict(list)
MIN_REID_MATCHES = 10        # Need 10 matching frames to confirm re-ID
SIMILARITY_THRESHOLD = 0.1  # Cosine similarity for re-identification
PRIMARY_ID = 1               # The ID you want to maintain for a specific person
last_seen = {}               # Track last appearance frames
assigned_reid_ids = {}       # Map DeepSort ID to a consistent Re-ID (your custom ID)
next_reid_id = 1             # Counter for new consistent Re-IDs

# Function to calculate cosine similarity
def cosine_similarity(a, b):
    # Ensure vectors are normalized if they are not already
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    if norm_a == 0 or norm_b == 0:
        return 0.0 # Handle division by zero
    return np.dot(a, b) / (norm_a * norm_b)

def process_video(input_path, output_path):
    global next_reid_id # Declare as global to modify in function
    global assigned_reid_ids # Declare as global
    global last_seen # Declare as global

    cap = cv2.VideoCapture(input_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    print(f"Video Info - Width: {width}, Height: {height}, FPS: {fps}")

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_count = 0

    # Store features of tracks that disappeared
    disappeared_track_features = {}

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        current_detections_boxes = [] # Store bounding boxes for overlap check

        try:
            results = model(frame,
                            classes=[0],
                            conf=0.7,
                            iou=0.45,
                            imgsz=640,
                            verbose=False)

            detections_for_deepsort = []
            for result in results:
                boxes = result.boxes.xyxy.cpu().numpy()
                confs = result.boxes.conf.cpu().numpy()
                for i, box in enumerate(boxes):
                    x1, y1, x2, y2 = map(int, box[:4])
                    conf = float(confs[i])
                    w, h = x2 - x1, y2 - y1

                    if w > 10 and h > 20: # Filter out very small detections
                        detections_for_deepsort.append(([x1, y1, w, h], conf, "person"))
                        current_detections_boxes.append((x1, y1, x2, y2)) # Add to for overlap check

            # Update DeepSort tracker
            tracks = tracker.update_tracks(detections_for_deepsort, frame=frame)

            # Active track IDs in the current frame (DeepSort's internal IDs)
            current_active_deepsort_ids = set()

            for track in tracks:
                if not track.is_confirmed():
                    continue

                deepsort_id = track.track_id
                ltrb = track.to_ltrb()

                # Check for overlap with current detections to identify potential "ghost" tracks
                # Removed 'continue' here to ensure features are collected even if detection is weak
                # This ensures consistent_id assignment and feature buffering for confirmed tracks
                has_overlap = False
                for (dx1, dy1, dx2, dy2) in current_detections_boxes:
                    if (ltrb[0] < dx2 and ltrb[2] > dx1 and
                        ltrb[1] < dy2 and ltrb[3] > dy1):
                        has_overlap = True
                        break

                # The track is confirmed by DeepSort. Proceed with processing its ID and features.
                current_active_deepsort_ids.add(deepsort_id)
                last_seen[deepsort_id] = frame_count # Update last seen frame

                # Re-identification Logic:
                current_feature = track.get_feature()

                if deepsort_id in assigned_reid_ids:
                    consistent_id = assigned_reid_ids[deepsort_id]
                else:
                    reidentified = False
                    best_match_id = None
                    max_similarity = -1

                    for prev_reid_id, features in disappeared_track_features.items():
                        for old_feature in features:
                            similarity = cosine_similarity(current_feature, old_feature)
                            if similarity > SIMILARITY_THRESHOLD and similarity > max_similarity:
                                max_similarity = similarity
                                best_match_id = prev_reid_id
                                reidentified = True
                                break
                        if reidentified:
                            break

                    if reidentified and best_match_id is not None:
                        consistent_id = best_match_id
                        assigned_reid_ids[deepsort_id] = consistent_id
                        if consistent_id in disappeared_track_features:
                            del disappeared_track_features[consistent_id] # Person is back, remove from disappeared
                        print(f"Re-identified DeepSort ID {deepsort_id} as consistent ID {consistent_id} (Similarity: {max_similarity:.2f})")
                    else:
                        consistent_id = next_reid_id
                        assigned_reid_ids[deepsort_id] = consistent_id
                        next_reid_id += 1
                        print(f"Assigned new consistent ID {consistent_id} to DeepSort ID {deepsort_id}")

                # Store features for potential future re-identification
                reid_buffer[consistent_id].append(current_feature)
                reid_buffer[consistent_id] = reid_buffer[consistent_id][-20:] # Keep last 20 features

                # Visualization
                color = (0, 255, 0) if consistent_id == PRIMARY_ID else (0, 0, 255) # Green for PRIMARY_ID, Red for others
                cv2.rectangle(frame,
                              (int(ltrb[0]), int(ltrb[1])),
                              (int(ltrb[2]), int(ltrb[3])),
                              color, 2)
                cv2.putText(frame, f"ID: {consistent_id}",
                           (int(ltrb[0]), int(ltrb[1])-10),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

            # Manage disappeared tracks for long-term re-ID
            for prev_deepsort_id in list(assigned_reid_ids.keys()):
                if prev_deepsort_id not in current_active_deepsort_ids:
                    consistent_id = assigned_reid_ids[prev_deepsort_id]

                    # If a DeepSort track has been gone for more than max_age, consider it truly disappeared
                    if frame_count - last_seen.get(prev_deepsort_id, 0) > 69: # max_age from tracker initialization
                        print(f"DeepSort ID {prev_deepsort_id} (Consistent ID {consistent_id}) truly disappeared.")

                        if consistent_id in reid_buffer and reid_buffer[consistent_id]:
                             if consistent_id not in disappeared_track_features:
                                 disappeared_track_features[consistent_id] = []
                             disappeared_track_features[consistent_id].extend(reid_buffer[consistent_id]) # Add all features from buffer
                             disappeared_track_features[consistent_id] = disappeared_track_features[consistent_id][-10:] # Keep last 10 for re-ID

                        del assigned_reid_ids[prev_deepsort_id]
                        if prev_deepsort_id in last_seen:
                            del last_seen[prev_deepsort_id]
                        # Optional: clear reid_buffer for this consistent_id if you want to save memory and rely only on disappeared_track_features
                        # if consistent_id in reid_buffer:
                        #     del reid_buffer[consistent_id]


        except Exception as e:
            print(f"\nError processing frame {frame_count}: {str(e)}")
            continue

        out.write(frame)

    cap.release()
    out.release()
    print("\nProcessing complete!")

In [3]:
print("Please upload your video file:")
# This will open a file upload dialog in Colab.
uploaded = files.upload()
# Get the name of the uploaded file.
input_video = list(uploaded.keys())[0]


Please upload your video file:


Saving harsh.mp4 to harsh.mp4


In [10]:
output_video = 'consistent_id_output.mp4'
# Call the main video processing function.
process_video(input_video, output_video)
# Download the processed video to your local machine.
files.download(output_video)

Video Info - Width: 1280, Height: 720, FPS: 59.51784706673364
Assigned new consistent ID 1 to DeepSort ID 1
Assigned new consistent ID 2 to DeepSort ID 2
Assigned new consistent ID 3 to DeepSort ID 3
DeepSort ID 2 (Consistent ID 2) truly disappeared.
Re-identified DeepSort ID 6 as consistent ID 2 (Similarity: 0.65)
DeepSort ID 1 (Consistent ID 1) truly disappeared.
DeepSort ID 3 (Consistent ID 3) truly disappeared.
Re-identified DeepSort ID 9 as consistent ID 1 (Similarity: 0.78)
Re-identified DeepSort ID 12 as consistent ID 3 (Similarity: 0.76)

Processing complete!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>